# imports

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold

import os
from Utils import cacheOnDisk
from queue.qp import qp,fakeqp
from addloglevels import sethandlers

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done
done
done
done


/usr/wisdom/python/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate


'12062018'

# Prepare X AND Y TABLES:

In [ ]:
datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
datasetName='PNP515_ss15000_rep2'

In [ ]:
# x is the MPA table, in this case, the species, capped to 0.0001 and binary table:
MBdf='MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_binary'
MBdfFolder='%s/MicrobiomeDataTables/FilteredAndMergedOnBD' %MyPath
file1='%s/%s' %(MBdfFolder,MBdf)
MB=pd.read_pickle(file1)
MB.head()

In [ ]:
# Yorig is one column of the TCR, not capped, trimmed, binary, shared by more than 20%:

#extract all relevant TCR seqs:
TCRdfName='sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_onlyProductiveTrue__percShared20_OLtrimmed_binary'
file2='%s/sharingAnalysis/%s' %(datasetFolder,TCRdfName)
TCRdf=pd.read_pickle(file2)
TCRdf.head()

In [ ]:
# find the most common sequence and take it to be Y

TCRdfSorted=TCRdf.copy()
TCRdfSorted.loc['sum',:]=TCRdfSorted.sum()
TCRdfSorted=TCRdfSorted.sort_values(by='sum',axis=1,ascending=False)
TCRdfSorted.tail()

In [ ]:
seq1data=pd.DataFrame(TCRdfSorted.iloc[:-1,0])
print len(seq1data)
seq1data[-5:]

In [ ]:
seq2data=pd.DataFrame(TCRdfSorted.iloc[:-1,1])
print len(seq2data)
seq2data[-5:]

In [ ]:
seq='CASSLSGSSYNEQFF'
seqData=pd.DataFrame(TCRdfSorted.loc[:,seq])
print seqData[-5:]
seqData=seqData.drop('sum',axis=0)
print seqData.shape
print seqData[-5:]

In [ ]:
print len(MB)

In [ ]:
# leave only common samples in each df (X and seq1data)
MB=MB.loc[[str(x) for x in MB.index.tolist() if x in seqData.index],:]
MB=MB.sort_index()
X=MB.copy()
print len(X)
print X.index[100]

In [ ]:
seqDatashort=seqData.loc[[str(x) for x in seqData.index.tolist() if x in X.index],:]
seqDatashort=seqDatashort.sort_index()
print len(seqDatashort)
print seqDatashort.index[100]

In [ ]:
oldColName=str(seqDatashort.columns.values[0])
print oldColName
Y=seqDatashort.rename(columns={oldColName:'Class'})
Y.head()

# old stuff

## run predictions:

In [ ]:
seq_MbDF_pairs=[('CASSLSGSSYNEQFF','MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_binary'),
                ('CASSLGQGAYEQYF','MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_binary'),
                ('CASSLAGSYEQYF','MPA_s_standardParams_capped0_0001_percShared5_OLtrimmed_binary')]


In [ ]:
pd.set_option('display.expand_frame_repr', False)


# generate X and Y and process them:

In [3]:
### prepare Y

datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
datasetName='PNP515_ss15000_rep2'
TCRdfName='sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_onlyProductiveTrue__percShared20_OLtrimmed_binary'
seq='CASSLAGSYEQYF'
YName=TCRdfName.replace('sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_onlyProductiveTrue__','TCR')
#extract TCR:
file2='%s/sharingAnalysis/%s' %(datasetFolder,TCRdfName)
TCRdf=pd.read_pickle(file2)
#extract seq data to be Y:
TCRdfSorted=TCRdf.copy()
TCRdfSorted.loc['sum',:]=TCRdfSorted.sum()
TCRdfSorted=TCRdfSorted.sort_values(by='sum',axis=1,ascending=False)
nSharedPerseq=TCRdfSorted.loc['sum',seq]

seqData=pd.DataFrame(TCRdfSorted.loc[:,seq])
print seqData.tail()
seqData=seqData.drop('sum',axis=0)
print 'seqData shape is %s_%s' %(seqData.shape[0],seqData.shape[1])
Y=seqData.copy()
print Y.tail()
print Y.sum()

        CASSLAGSYEQYF
Sample               
BD944             0.0
BD945             1.0
BD948             0.0
BD949             0.0
sum              98.0
seqData shape is 392_1
        CASSLAGSYEQYF
Sample               
BD943             0.0
BD944             0.0
BD945             1.0
BD948             0.0
BD949             0.0
CASSLAGSYEQYF    98.0
dtype: float64


In [5]:
# PREPARE x
folder='%s/MicrobiomeDataTables/MbFeatureTables' %MyPath
tabType='notFiltered'

file2='%s/%s_mergedFeatureTable' %(folder,tabType)
mergedFeatureTable=pd.read_pickle(file2)

X=mergedFeatureTable.copy()
XName='mergedFeatureTable_%s' %tabType

X.head()

,nTaxa_MPAgclonDivFeatures,meanRA_MPAgclonDivFeatures,meanRAno0_MPAgclonDivFeatures,meadianRAno0_MPAgclonDivFeatures,stdRA_MPAgclonDivFeatures,stdRAno0_MPAgclonDivFeatures,maxRA_MPAgclonDivFeatures,max/meanRA_MPAgclonDivFeatures,max/meanRAno0_MPAgclonDivFeatures,sumTop10_MPAgclonDivFeatures,...,meadianRAno0_MPAsclonDivFeatures,stdRA_MPAsclonDivFeatures,stdRAno0_MPAsclonDivFeatures,maxRA_MPAsclonDivFeatures,max/meanRA_MPAsclonDivFeatures,max/meanRAno0_MPAsclonDivFeatures,sumTop10_MPAsclonDivFeatures,shannon_MPAsclonDivFeatures,simpson_MPAsclonDivFeatures,berger_parker_d_MPAsclonDivFeatures
BD,,,,,,,,,,,,,,,,,,,,,
BD1,38.0,0.003683,0.026312,0.007379,0.022636,0.056806,0.327642,88.962774,12.452272,0.831991,...,0.003768,0.007606,0.022070,0.130465,105.880228,10.808238,0.590882,4.921961,0.947537,0.131446
BD10,37.0,0.003408,0.024952,0.007728,0.017608,0.042841,0.186990,54.872922,7.493872,0.795301,...,0.003070,0.008200,0.026818,0.186990,167.027007,14.066920,0.595756,4.488300,0.924755,0.210586
BD101,39.0,0.003682,0.025636,0.004835,0.017157,0.039628,0.133763,36.325610,5.217860,0.803310,...,0.004448,0.007854,0.023733,0.128536,105.977308,10.045463,0.633085,4.815540,0.941811,0.132990
BD105,37.0,0.003683,0.027019,0.010289,0.018647,0.045053,0.189747,51.523432,7.022749,0.826908,...,0.004267,0.007171,0.020860,0.104872,84.925090,8.143240,0.571824,4.936572,0.952547,0.106816
BD106,28.0,0.003686,0.035709,0.004640,0.035875,0.109642,0.585405,158.797841,16.393941,0.937396,...,0.002706,0.020073,0.075197,0.583773,468.515676,35.040865,0.828480,2.950158,0.648735,0.584585


In [6]:
#co-processing of X and Y

X=X.loc[[str(x) for x in X.index.tolist() if x in Y.index],:]
X=X.sort_index()
print 'X shape is %s_%s' %(X.shape[0], X.shape[1])
print 'the 100th sample in X is %s' %X.index[100]
print X.iloc[:3,:3]
Y=Y.loc[[str(x) for x in Y.index.tolist() if x in X.index],:]
Y=Y.sort_index()
oldColName=str(Y.columns.values[0])
Y=Y.rename(columns={oldColName:'Class'})
print 'Y shape is %s_%s' %(Y.shape[0], Y.shape[1])
print 'the 100th sample in Y is %s' %Y.index[100]
print Y.head(3)

X shape is 371_2316
the 100th sample in X is BD375
       nTaxa_MPAgclonDivFeatures  meanRA_MPAgclonDivFeatures  \
BD                                                             
BD1                         38.0                    0.003683   
BD10                        37.0                    0.003408   
BD101                       39.0                    0.003682   

       meanRAno0_MPAgclonDivFeatures  
BD                                    
BD1                         0.026312  
BD10                        0.024952  
BD101                       0.025636  
Y shape is 371_1
the 100th sample in Y is BD375
        Class
Sample       
BD1       1.0
BD10      0.0
BD101     0.0


# develop train_test_split design:

In [ ]:
from sklearn.model_selection import train_test_split  
# seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, stratify=Y)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
print y_pred
predictions = [round(value) for value in y_pred]
print predictions
# evaluate predictions
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Accuracy2: %.2f%%" % (accuracy2 * 100.0))

# improve CV splitting development:

In [ ]:
useCV=True
stratifiedCV=False
n_splits=3
model= lgb.LGBMClassifier
model_params={'n_estimators':20, 'learning_rate': 0.1,'max_depth':3,'numthreads':2}

##### copy from here to the function####
#convert X and Y to arrays so they will work with the StratifiedKFold method
X1=np.array(X.values)
Y1=np.array(Y.values)
Y1=np.reshape(Y1,[Y1.shape[0],])
print X1.shape
print Y1.shape

if useCV:
    print 'splitting train_test using cross validation with %s splits...' %n_splits
#     groups = np.array(range(X1.shape[0]))
    if stratifiedCV:
        kf = StratifiedKFold(n_splits=n_splits)
    else:
        kf = KFold(n_splits=n_splits)

    y_pred_df = pd.DataFrame(index=Y.index, columns=['pred_proba','y_pred'])
    i=0
    for train_index,val_index in kf.split(X1, Y1):
        print i
        i+=1
        X_train, X_val = X1[train_index], X1[val_index]
        y_train, y_val = Y1[train_index], Y1[val_index]
        
        print 'y_train mean=%s' %y_train.mean()
        print 'y_val mean=%s' %y_val.mean()

        # creating the model object
        m = model(**model_params)

        # fitting the training
        print 'fitting the model'
        m.fit(X_train, y_train,
                eval_set=[(X_val, y_val)],
                early_stopping_rounds=None, verbose=-1)
        # getting the predictions for the test
        print 'getting predictions....'
        y_pred_proba = m.predict_proba(X_val)
        predictions = m.predict(X_val)
        inds=[x for x in range(len(Y.index)) if x in val_index]

        y_pred_df.iloc[inds, :] = np.expand_dims(y_pred_proba[:,1], 1)
        y_pred_df.iloc[inds, 1] = predictions
        
        print 'y_pred_df_head:'
        print y_pred_df.head()

# develop: gridSearchCV:
THIS PART SHOULD BE A SEPERATE FUNCTION:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

model= lgb.LGBMClassifier
#convert X and Y to arrays so they will work with the StratifiedKFold method
X1=np.array(X.values)
Y1=np.array(Y.values)
Y1=np.reshape(Y1,[Y1.shape[0],])
print X1.shape
print Y1.shape

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X1, Y1, test_size=0.5, random_state=0)
print X_train
print y_train

# Set the parameters by cross-validation
tuned_parameters =[{'n_estimators':[20,100,1000], 'learning_rate': [0.01,0.05,0.1,1],'max_depth':[-1,3,5]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(model(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score,n_jobs=32)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

(371, 2316)
(371,)
[[2.90000000e+01 3.68592950e-03 3.44754621e-02 ... 4.65342738e+00
  9.37595017e-01 1.45896657e-01]
 [3.70000000e+01 3.67845980e-03 2.69868061e-02 ... 4.90275927e+00
  9.51480783e-01 1.03377687e-01]
 [3.60000000e+01 3.24589436e-03 2.43932953e-02 ... 4.96175612e+00
  9.53365480e-01 1.11111111e-01]
 ...
 [3.80000000e+01 3.63969736e-03 2.59956807e-02 ... 5.20228919e+00
  9.61758432e-01 8.91393443e-02]
 [3.20000000e+01 3.68424654e-03 3.12381418e-02 ... 2.54579389e+00
  5.82526273e-01 6.39393939e-01]
 [4.00000000e+01 3.62233774e-03 2.45802473e-02 ... 4.82329050e+00
  9.50170762e-01 1.06122449e-01]]
[0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 

# develop - 

In [ ]:
model= lgb.LGBMClassifier
#convert X and Y to arrays so they will work with the StratifiedKFold method
X1=np.array(X.values)
Y1=np.array(Y.values)
Y1=np.reshape(Y1,[Y1.shape[0],])
print X1.shape
print Y1.shape


original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                   'min_samples_split': 5}

X_train, X_test, y_train, y_test = train_test_split(
    X1, Y1, test_size=0.33, random_state=0)
plt.figure()

for label, color, setting in [('No shrinkage', 'orange',
                               {'learning_rate': 1.0, 'subsample': 1.0}),
                              ('learning_rate=0.1', 'turquoise',
                               {'learning_rate': 0.1, 'subsample': 1.0}),
                              ('subsample=0.5', 'blue',
                               {'learning_rate': 1.0, 'subsample': 0.5}),
                              ('learning_rate=0.1, subsample=0.5', 'gray',
                               {'learning_rate': 0.1, 'subsample': 0.5}),
                              ('learning_rate=0.1, max_features=2', 'magenta',
                               {'learning_rate': 0.1, 'max_features': 2})]:
    
    params = dict(original_params)
    params.update(setting)

    clf = ensemble.GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)

    # compute test set deviance
    test_deviance = np.zeros((params['n_estimators'],), dtype=np.float64)

    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        # clf.loss_ assumes that y_test[i] in {0, 1}
        test_deviance[i] = clf.loss_(y_test, y_pred)

    plt.plot((np.arange(test_deviance.shape[0]) + 1)[::5], test_deviance[::5],
            '-', color=color, label=label)

plt.legend(loc='upper left')
plt.xlabel('Boosting Iterations')
plt.ylabel('Test Set Deviance')

plt.show()

# define function to use any model and parameters:

In [ ]:
def predictBinaryByDistMat(Y,YName,X,XName,ResultFolder,model,model_name,model_params,n_splits,useCV=True,stratifiedCV=True):
    
    #(1) arrange result folders:
    print 'arranging  result folders:'
    predResultsDF=pd.DataFrame()
    
    predResultsDFFolder='%s/predictionDFs' %ResultFolder #define folder for all result dfs in this model
    if not isdir(predResultsDFFolder):
        makedirs(predResultsDFFolder)
        print 'generating predResultsDFFolder %s' %predResultsDFFolder
        
    predResultsfigFolder='%s/figs' %ResultFolder #define folder for figs in this model
    if not isdir(predResultsfigFolder):
        makedirs(predResultsfigFolder)
        print 'predResultsfigFolder %s' %predResultsfigFolder

    #generate result DF name based on model_params:
    d=OrderedDict(sorted(model_params.items(), key=lambda t: t[0]))
    try:
        d2=d.copy()
        del d2['numthreads']
    except:
        d2=d.copy()
    predResultsDFName='_'.join(['%s%s' %(key.replace('_',''), value) for (key, value) in d2.items()]) #generate a file name based on params
    predResultsDFName=predResultsDFName.replace('.','-')
    if useCV:
        predResultsDFName='%s_CV%s' %(predResultsDFName,n_splits)
    
    predResultsDFfile='%s/%s' %(predResultsDFFolder,predResultsDFName)
    existingDFs=[f for f in listdir( predResultsDFFolder) if isfile(join(predResultsDFFolder, f))]
    

    if predResultsDFName not in existingDFs:  
        

        # common processing of X and y:
        # leave only common samples in each df (X and seq1data)
        X=X.loc[[str(x) for x in X.index.tolist() if x in Y.index],:]
        X=X.sort_index()
        print 'X shape is %s_%s' %(X.shape[0], X.shape[1])
        print 'the 100th sample in X is %s' %X.index[100]
        print X.iloc[:3,:3]
        Y=Y.loc[[str(x) for x in Y.index.tolist() if x in X.index],:]
        Y=Y.sort_index()
        oldColName=str(Y.columns.values[0])
        Y=Y.rename(columns={oldColName:'Class'})
        print 'Y shape is %s_%s' %(Y.shape[0], Y.shape[1])
        print 'the 100th sample in Y is %s' %Y.index[100]
        print Y.head(3)

        #(2) model fitting and predictions:

        
        if useCV:
            print 'splitting train_test using cross validation with %s splits...' %n_splits
            groups = np.array(range(X.shape[0]))
            if stratifiedCV:
                group_kfold = StratifiedKFold(n_splits=n_splits)
            else:
                group_kfold = GroupKFold(n_splits=n_splits)
        
            y_pred_df = pd.DataFrame(index=Y.index, columns=['pred_proba'])
            i=0
            for train_index, val_index in group_kfold.split(X, Y, groups):
                print i
                i+=1
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = Y.loc[X_train.index].Class, Y.loc[X_val.index].Class

                # creating the model object
                m = model(**model_params)

                # fitting the training
                m.fit(X_train, y_train,
                        eval_set=[(X_val, y_val)],
                        early_stopping_rounds=None, verbose=-1)
                # getting the predictions for the test
                y_pred_proba = m.predict_proba(X_val)
                y_pred_df.loc[y_val.index, :] = np.expand_dims(y_pred_proba[:,1], 1)
        else:
            test_size = 0.33
            print 'using normal train_test split with test_size=%s' %test_size
            from sklearn.model_selection import train_test_split  
            # seed = 7
            
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, stratify=Y)
            print 'fitting model...'
            model = model
            model.fit(X_train, y_train)
            # make predictions for test data
            print 'predicting...'
            y_pred = model.predict(X_test)
            predictions = [round(value) for value in y_pred]
            # evaluate predictions
            from sklearn.metrics import accuracy_score
            accuracy = accuracy_score(y_test, y_pred)
            print("Accuracy: %.2f%%" % (accuracy * 100.0))
            y_pred_df = pd.DataFrame(index=Y.index, data={'pred':y_pred})

        #(4) plots:
        print 'generating plots...'

        # this plot shows the probabilities returned by the predictor colored by the class
        plt.figure(figsize=(3,2))
        plt.scatter(range(y_pred_df.shape[0]), y_pred_df.pred_proba, c=Y.Class)
        plt.show()

        # plot ROC and PR curves

        from sklearn import metrics
        fpr, tpr, thresholds = metrics.roc_curve(Y.Class+1, y_pred_df.pred_proba, pos_label=2)
        roc_auc = metrics.auc(fpr, tpr)
        print roc_auc

        plt.figure(figsize=(20,7))
        plt.subplot(1,2,1)
        lw = 2
        plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', fontsize=15)
        plt.ylabel('True Positive Rate', fontsize=15)
        plt.title('ROC curve - ' , fontsize=20)
        plt.legend(loc="lower right", fontsize=15)

        plt.subplot(1,2,2)
        precision, recall, _ = metrics.precision_recall_curve(Y.Class, y_pred_df.pred)
        pr_auc = metrics.auc(recall, precision)

        plt.step(recall, precision, color='b', alpha=0.2, where='post')
        plt.fill_between(recall, precision, step='post', alpha=0.5,
                         color='darkorange', label='Precision Recall curve - AUC = {0:0.3f}'.format(pr_auc))
        plt.plot([0, 1], [Y.sum()/Y.shape[0], Y.sum()/Y.shape[0]], color='navy', lw=lw, linestyle='--')
        plt.xlabel('Recall', fontsize=15)
        plt.ylabel('Precision', fontsize=15)
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title('Precision Recall curve', fontsize=20)
        plt.legend(loc="upper right", fontsize=15)

        if roc_auc>0.55:
            predResultsFigfile='%s/%s_ROC-PR.png' %(predResultsfigFolder,predResultsDFName)
            plt.savefig(predResultsFigfile, bbox_inches='tight')

        plt.show()

        #(5) generate summarizing df:
        print 'generating summarizing df'
        predResultsDF.loc[0,'Yname']=YName
        predResultsDF.loc[0,'Xname']=XName
        preResultsDF.loc[0,'model_name']=model_name
        predResultsDF.loc[0,'roc_auc']=round(roc_auc,3)
        predResultsDF.loc[0,'pr_auc']=round(pr_auc,3)
        predResultsDF.loc[0,'useCV']=useCV
        if useCV:
            predResultsDF.loc[0,'n_splits']=n_splits
        
        
        for (key, value) in OrderedDict(sorted(model_params.items(), key=lambda t: t[0])).items():
            predResultsDF.loc[0,key]=value
        
        nPos=int(Y.Class.sum())
        nNeg=len(Y)-nPos
        
        print nPos, nNeg

        predResultsDF.loc[0,'nPos']=nPos
        predResultsDF.loc[0,'nNeg']=nNeg
        
        
        predResultsDF.to_pickle(predResultsDFfile)
        print predResultsDFfile
        
    else:
        print 'this prediction already exists'
    
    return predResultsDF
    
    
    
    
    
    
    

## run with default parameters:

In [ ]:
### prepare seqData

datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
datasetName='PNP515_ss15000_rep2'
TCRdfName='sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_onlyProductiveTrue__percShared20_OLtrimmed_binary'
seq='CASSLAGSYEQYF'
YName=TCRdfName.replace('sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_onlyProductiveTrue__','TCR')
#extract TCR:
file2='%s/sharingAnalysis/%s' %(datasetFolder,TCRdfName)
TCRdf=pd.read_pickle(file2)
#extract seq data to be Y:
TCRdfSorted=TCRdf.copy()
TCRdfSorted.loc['sum',:]=TCRdfSorted.sum()
TCRdfSorted=TCRdfSorted.sort_values(by='sum',axis=1,ascending=False)
nSharedPerseq=TCRdfSorted.loc['sum',seq]

seqData=pd.DataFrame(TCRdfSorted.loc[:,seq])
print seqData.tail()
seqData=seqData.drop('sum',axis=0)
print 'seqData shape is %s_%s' %(seqData.shape[0],seqData.shape[1])
Y=seqData.copy()
print Y.tail()
print Y.sum()

In [ ]:
file1='%s/%s' %(MBdfFolder,MBdfName)
MB=pd.read_pickle(file1)
X=MB.copy()

#extract TCR:
file2='%s/sharingAnalysis/%s' %(datasetFolder,TCRdfName)
TCRdf=pd.read_pickle(file2)
#extract seq data to be Y:
TCRdfSorted=TCRdf.copy()
TCRdfSorted.loc['sum',:]=TCRdfSorted.sum()
TCRdfSorted=TCRdfSorted.sort_values(by='sum',axis=1,ascending=False)
nSharedPerseq=TCRdfSorted.loc['sum',seq]

seqData=pd.DataFrame(TCRdfSorted.loc[:,seq])
print seqData.tail()
seqData=seqData.drop('sum',axis=0)
print 'seqData shape is %s_%s' %(seqData.shape[0],seqData.shape[1])
Y=seqData.copy()
print Y.tail()
print Y.sum()

In [ ]:
ResultFolder='%s/TCR_mb_results/seqTCRpredictions' %datasetFolder
model= lgb.LGBMClassifier
model_name='LGBMClassifier'
n_splits=5
model_params={}
predResultsDF=predictBinaryByDistMat(Y,YName,X,XName,ResultFolder,model,model_name,model_params,n_splits=n_splits,useCV=True)

In [ ]:
ResultFolder='%s/TCR_mb_results/seqTCRpredictions' %datasetFolder
model= lgb.LGBMClassifier
model_name='LGBMClassifier'
learning_rate_list=[0.01,0.05,0.1,0.2]
n_estimators_list=[10,20,50,100]
max_depth_list=[-1,2,3,4]
n_splits_list=[5,10]

count=0
for learning_rate in learning_rate_list:
    for n_estimators in n_estimators_list:
        for max_depth in max_depth_list:
            for n_splits in n_splits_list:
                model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}

                print count,learning_rate,n_estimators, max_depth, n_splits

                predResultsDF=predictBinaryByDistMat(Y,YName,X,XName,ResultFolder,model,model_name,model_params,n_splits=n_splits,useCV=True)
                
                count=count+1

In [ ]:
predResultsDF

## get results:

In [ ]:
modelResultFolder='%s/%s' %(ResultFolder,model_name) #define folder for all model results
predResultsDFFolder='%s/predictionDFs' %modelResultFolder #define folder for all result dfs in this model

predResults_lgb=concat_summarizing_dfs(predResultsDFFolder)

In [ ]:
pd.set_option('display.expand_frame_repr', False)
predResults_lgb=predResults_lgb.sort_values(by='roc_auc',ascending=False)
predResults_lgb

## run with many features:

### prepare x and y:

In [ ]:
### prepare seqData

datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
datasetName='PNP515_ss15000_rep2'
TCRdfName='sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_onlyProductiveTrue__percShared20_OLtrimmed_binary'
seq='CASSLAGSYEQYF'
YName=TCRdfName.replace('sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_onlyProductiveTrue__','TCR')
#extract TCR:
file2='%s/sharingAnalysis/%s' %(datasetFolder,TCRdfName)
TCRdf=pd.read_pickle(file2)
#extract seq data to be Y:
TCRdfSorted=TCRdf.copy()
TCRdfSorted.loc['sum',:]=TCRdfSorted.sum()
TCRdfSorted=TCRdfSorted.sort_values(by='sum',axis=1,ascending=False)
nSharedPerseq=TCRdfSorted.loc['sum',seq]

seqData=pd.DataFrame(TCRdfSorted.loc[:,seq])
print seqData.tail()
seqData=seqData.drop('sum',axis=0)
print 'seqData shape is %s_%s' %(seqData.shape[0],seqData.shape[1])
Y=seqData.copy()
print Y.tail()
print Y.sum()

In [ ]:
folder='%s/MicrobiomeDataTables/MbFeatureTables' %MyPath
tabType='notFiltered'

file2='%s/%s_mergedFeatureTable' %(folder,tabType)
mergedFeatureTable=pd.read_pickle(file2)

X=mergedFeatureTable.copy()
XName='mergedFeatureTable_%s' %tabType

X.head()

### run model:

In [ ]:
ResultFolder='%s/TCR_mb_results/seqTCRpredictions/manyFeaturesNotFiltered' %datasetFolder
model= lgb.LGBMClassifier
model_name='LGBMClassifier'
# learning_rate_list=[0.01,0.05,0.1,0.2]
# n_estimators_list=[10,20,50,100]
# max_depth_list=[-1,2,3,4]
# n_splits_list=[5,10]

learning_rate=0.1
n_estimators=100
max_depth=3
n_splits=10

model_params={'learning_rate': learning_rate, 'n_estimators': n_estimators,'max_depth': max_depth,'num_threads':2}
predResultsDF=predictBinaryByDistMat(Y,YName,X,XName,ResultFolder,model,model_name,model_params,n_splits=n_splits,useCV=True)
